In [6]:
pip install pyodbc 

In [7]:
import numpy as np
import pandas as pd
import pyodbc

In [17]:
#CREATE A CONNECTION TO MY DATABASE


#DATABASE DETAILS
server = 'Ajay\\AJAY'  
database = 'CampusX'
username = 'sa'
password = 'CampusX@21'

#TABLES_NAME
table_names = ['company_dim2','job_postings_fact2','skills_dim2','skills_job_dim2']
#CONNECTION STRING
conn_str =  f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

#CONNECT TO MS-SQL SERVER
conn= pyodbc.connect(conn_str)
print("Connection Successfull")

#DICTIONARY TO STORE ALL THE TABLES
dataframes= {}

for table_name in table_names:
    query= f"Select * from {table_name}"
    df= pd.read_sql(query, conn)
    dataframes[table_name]= df

conn.close()

Connection Successfull


C:\Users\acer\AppData\Local\Temp\ipykernel_11728\134772098.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df= pd.read_sql(query, conn)


In [18]:
company= dataframes['company_dim2']
Job_posting= dataframes['job_postings_fact2']
skill_id = dataframes['skills_job_dim2']
skills = dataframes['skills_dim2']

In [25]:
merge = pd.merge(company, Job_posting, on="company_id", how= "left")

In [27]:
merge = pd.merge(merge, skill_id, on="job_id", how= "left")

In [29]:
merge = pd.merge(merge, skills, on="skill_id", how= "left")

In [32]:
merge.columns

Index(['company_id', 'name', 'link', 'link_google', 'thumbnail', 'job_id',
       'job_title_short', 'job_title', 'job_location', 'job_via',
       'job_schedule_type', 'job_work_from_home', 'search_location',
       'job_posted_date', 'job_no_degree_mention', 'job_health_insurance',
       'job_country', 'salary_rate', 'salary_year_avg', 'salary_hour_avg',
       'skill_id', 'skills', 'type'],
      dtype='object')

In [36]:
columns_to_be_dropped = ['link', 'link_google', 'thumbnail','job_via','job_no_degree_mention', 'job_health_insurance',
       'job_country', 'salary_rate', 'salary_year_avg', 'salary_hour_avg',
       'skill_id','type']

In [37]:
merge.drop(columns=columns_to_be_dropped,inplace=True)

In [38]:
merge.columns

Index(['company_id', 'name', 'job_id', 'job_title_short', 'job_title',
       'job_location', 'job_schedule_type', 'job_work_from_home',
       'search_location', 'job_posted_date', 'skills'],
      dtype='object')

In [43]:
merge['skills']= merge['skills'].fillna('').astype(str)

In [47]:
final_dataframe = merge.groupby(['company_id', 'name', 'job_id', 'job_title_short', 'job_title',
       'job_location', 'job_schedule_type', 'job_work_from_home',
       'search_location', 'job_posted_date'])['skills'].agg(','.join).reset_index()

In [48]:
final_dataframe

,company_id,name,job_id,job_title_short,job_title,job_location,job_schedule_type,job_work_from_home,search_location,job_posted_date,skills
0,0,Cryptology,0,Data Analyst,Marketing Data Analyst,Anywhere,Full-time,True,Serbia,2023-09-25 17:46:06,"sql,python,excel,tableau,power bi"
1,0,Cryptology,923232,Data Analyst,Marketing Data Analyst,Anywhere,Full-time,True,Spain,2023-09-25 17:51:21,"sql,python,excel,tableau,power bi"
2,0,Cryptology,1695656,Data Analyst,Marketing Data Analyst,Anywhere,Full-time,True,United Kingdom,2023-09-25 18:11:19,"sql,python,excel,tableau,power bi"
3,0,Cryptology,1819517,Data Analyst,Marketing Data Analyst,"Madrid, Spain",Full-time,False,Spain,2023-10-10 10:32:19,"sql,python,excel,tableau,power bi"
4,1,Edraak,55,Cloud Engineer,Storage and Virtualization Engineer,"Kuwait City, Kuwait",Full-time,False,Kuwait,2023-07-30 17:49:18,vmware
...,...,...,...,...,...,...,...,...,...,...,...
773931,787652,Rotary Engineering Pte. Ltd.,1424742,Senior Data Scientist,Senior Design Engineer (Instrumentation),Singapura,Kontraktor,False,Singapore,2023-03-13 06:16:16,"sheets,excel,word,terminal"
773932,787674,Pure App,549918,Senior Data Engineer,Senior Data Engineer,"Berlin, Jerman",Pekerjaan tetap,False,Germany,2023-03-13 06:19:07,"sql,python,bigquery,aws,airflow,fastapi,github..."
773933,787675,Staff-UP Consulting Group,782857,Business Analyst,PreSales Engineer,"Almaty, Kazakhstan",Pekerjaan tetap,False,Kazakhstan,2023-03-12 06:31:13,windows
773934,787680,CAREERSTAR INTERNATIONAL PTE. LTD.,828965,Software Engineer,DevOps Engineer,Singapura,Pekerjaan tetap,False,Singapore,2023-03-13 06:16:16,"bash,python,perl,linux,unix,kubernetes,docker,..."
